In [3]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,k):
    '''
    计算二维互相关运算  卷积
    :param X: 输入张量
    :param k: 卷积核
    :return:
    '''
    h,w=k.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*k).sum()
    return Y

In [4]:
#验证一下
X=torch.tensor([[0.0,1.0,2.0],[3.,4.,5.],[6.,7.,8.]])
k=torch.tensor([[0.,1.],[2.,3.]])

corr2d(X,k)

tensor([[19., 25.],
        [37., 43.]])

基于上面定义的corr2d函数实现二维卷积层。在__init__构造函数中，将weight和bias声明为两个模型参数。前向传播函数调用corr2d函数并添加偏置。

In [5]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))

    def forward(self,X):
        return corr2d(X,self.weight)+self.bias


如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。 首先，我们构造一个像素的黑白图像。中间四列为黑色（0），其余像素为白色（1）

In [6]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [7]:
K=torch.tensor([[1.,-1.]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

这个卷积核K只可以检测垂直边缘，无法检测水平边缘。输入的二维图像转置，再进行如上的互相关运算

In [8]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

6.2.4. 学习卷积核
 我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置

In [16]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
## 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核

In [18]:

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X=X.reshape((1,1,6,8))
Y=Y.reshape(1,1,6,7)
lr=3e-2 #learning-rate
for i in range(10):
    Y_hat=conv2d(X)
    loss=(Y_hat-Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    #迭代卷积核
    conv2d.weight.data[:]-=lr*conv2d.weight.grad
    if (i+1)%2==0:
        print(f'epoch {i+1},loss {loss.sum():.3f}')

epoch 2,loss 9.759
epoch 4,loss 2.882
epoch 6,loss 0.993
epoch 8,loss 0.375
epoch 10,loss 0.148


In [15]:
conv2d.weight.data.reshape((1,2))

tensor([[ 1.2321, -0.7357]])

由于卷积核是从数据中学习到的，因此无论这些层执行严格的卷积运算还是互相关运算，卷积层的输出都不会受到影响。

应用多层卷积时，我们常常丢失边缘像素。 由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。 但随着我们应用许多连续卷积层，累积丢失的像素数就多了。 解决这个问题的简单方法即为填充（padding）：在输入图像的边界填充元素（通常填充元素是）0

卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。 选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列